In [2]:
pip install openai

In [3]:
import os
from openai import AzureOpenAI

endpoint = "https://s2246-mf8dy9aa-eastus2.cognitiveservices.azure.com/"
model_name = "gpt-4.1"
deployment = "gpt-4.1"

subscription_key = "A8RcAH9JqUIjSgra5D0PgzTlU1K0iElzCwpqVTrNfdA7c0HJb7DbJQQJ99BIACHYHv6XJ3w3AAAAACOGYaJC"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)


# Task
Enable the selected model to perform Zero-Shot Learning (ZSL) for Feedback Generation and Few-Shot Learning (FSL) for AI Detection using the data in "/content", evaluate outputs according to established criteria, and document the process, results, and observations, including the lack of examples for ZSL Feedback Generation.

## Load data

### Subtask:
Load the data from the JSON files in the `/content` directory.


In [4]:
import json
import os

data = {}

In [5]:
for file_name in os.listdir('/content'):
    if file_name.endswith('.json') and os.path.isfile(os.path.join('/content', file_name)):
        with open(os.path.join('/content', file_name), 'r') as f:
            key = os.path.splitext(file_name)[0]
            data[key] = json.load(f)

display(data)

{'accounting': {'domain': 'Accounting',
  'prompt': 'Analyze the impact of blockchain technology on traditional accounting practices, including auditing and financial reporting.',
  'rubric': {'rubric_id': 'rub_accounting_0001',
   'criteria': [{'criterion_id': 'c1',
     'name': 'Understanding of Blockchain Concepts',
     'description': 'Demonstrates knowledge of blockchain principles (decentralization, immutability, smart contracts) and their relevance to accounting.',
     'performance_descriptors': {'excellent': 'Clearly explains multiple blockchain concepts and maps each to relevant accounting applications.',
      'good': 'Accurately describes 1–2 blockchain concepts and gives examples.',
      'average': 'Mentions blockchain concepts but with limited detail.',
      'needs_improvement': 'Basic or unclear understanding of concepts.',
      'poor': 'Fails to explain blockchain accurately.'}},
    {'criterion_id': 'c2',
     'name': 'Analysis of Accounting Impact',
     'descripti

In [6]:
for domain, domain_data in data.items():
    prompt = domain_data['prompt']
    rubric = domain_data['rubric']

    for submission in domain_data['submissions']:
        submission_text = submission['final_submission']

        zsl_prompt = f"""Domain: {domain_data['domain']}
Prompt given to student: {prompt}
Student Submission: {submission_text}
Rubric: {rubric}

Generate feedback for the student submission based on the provided rubric.
"""
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that generates feedback for student submissions based on a rubric."},
                    {"role": "user", "content": zsl_prompt}
                ]
            )
            feedback = response.choices[0].message.content
            print(f"Generated feedback for {domain} submission:")
            print(feedback)
            submission['zsl_feedback'] = feedback
        except Exception as e:
            print(f"Error generating feedback for {domain} submission: {e}")
            submission['zsl_feedback'] = f"Error generating feedback: {e}"

display(data)

Generated feedback for accounting submission:
Feedback for Student Submission:

Criterion 1: Understanding of Blockchain Concepts  
Excellent. Your response demonstrates a clear and precise understanding of core blockchain concepts, including decentralization, immutability, and smart contracts. You explicitly explain each principle and thoughtfully map them to accounting applications—such as how smart contracts can automate invoicing and reconcile tasks, and the role of decentralized ledgers in creating transparency.

Criterion 2: Analysis of Accounting Impact  
Excellent. You offer insightful analysis regarding the impact of blockchain on auditing and financial reporting. The discussion encompasses benefits such as enhanced transparency, tamper-proof audit trails, real-time reporting, and improved fraud detection. You also address important challenges including regulatory uncertainty, scalability issues, and integration costs, showing a balanced grasp of both advantages and limitation

{'accounting': {'domain': 'Accounting',
  'prompt': 'Analyze the impact of blockchain technology on traditional accounting practices, including auditing and financial reporting.',
  'rubric': {'rubric_id': 'rub_accounting_0001',
   'criteria': [{'criterion_id': 'c1',
     'name': 'Understanding of Blockchain Concepts',
     'description': 'Demonstrates knowledge of blockchain principles (decentralization, immutability, smart contracts) and their relevance to accounting.',
     'performance_descriptors': {'excellent': 'Clearly explains multiple blockchain concepts and maps each to relevant accounting applications.',
      'good': 'Accurately describes 1–2 blockchain concepts and gives examples.',
      'average': 'Mentions blockchain concepts but with limited detail.',
      'needs_improvement': 'Basic or unclear understanding of concepts.',
      'poor': 'Fails to explain blockchain accurately.'}},
    {'criterion_id': 'c2',
     'name': 'Analysis of Accounting Impact',
     'descripti

In [16]:
if data: # Check if data is loaded
    # Generate synthetic examples for FSL outside the dataset
    fsl_examples = [
        {"final_submission": "I found a Hand written letter in my grandmother's attic. It's a bit faded but you can still read most of it. The letter consists of the memories she made with our family and how much she loves me as her only grandchild.", "label_type": "Human"},
        {"final_submission": "Yesterday I went to Queen Victoria Market with my roommates. We tried Turkish delight, and I bought some groceries for my room. It was such a good day, i enjoyed alot.", "label_type": "Human"},
        {"final_submission": "I missed my train this morning because I did'nt heard the alarm, and then it started raining as I walked to uni. Honestly, Mondays are annoying.", "label_type": "Human"},
        {"final_submission": "Artificial intelligence has transformed multiple industries, including healthcare, finance, and education, by enabling automation, predictive analytics, and personalized recommendations.", "label_type": "AI"},
        {"final_submission": "The rapid advancement of machine learning models highlights the importance of ethical considerations, including bias mitigation, explainability, and fairness across diverse populations.", "label_type": "AI"},
        {"final_submission": "The impact of climate change on global ecosystems is a complex issue with far-reaching consequences. Scientific models predict a significant increase in average global temperatures, leading to rising sea levels, more frequent extreme weather events, and disruptions to biodiversity. While some argue for immediate, drastic measures to reduce carbon emissions, others emphasize the need for technological innovation and adaptation strategies.", "label_type": "AI"},
        {"final_submission": "I was so nervous before my job interview, but once I started talking it went smoothly. Communication is a fundamental skill that helps establish trust and credibility in professional environments.", "label_type": "Hybrid"},
        {"final_submission": "Last night I cooked pasta for the first time and it actually turned out delicious. Cooking at home is also an effective way to save money and improve overall nutritional health.", "label_type": "Hybrid"},
        {"final_submission": "I love playing cricket with my friends on weekends. Team sports are widely recognized for fostering collaboration, leadership, and strategic thinking.", "label_type": "Hybrid"}
    ]


    # Shuffle examples to avoid bias based on order
    import random
    random.shuffle(fsl_examples)

    # Iterate through all domains and their submissions
    all_submissions = []
    for domain_data in data.values():
        all_submissions.extend(domain_data.get('submissions', []))

    for submission in all_submissions:
        submission_text = submission['final_submission']

        messages = [{"role": "system", "content": "You are an AI that classifies text as either Human, Hybrid, or AI generated. Respond only with 'Human', 'Hybrid', or 'AI'."}]

        # Add FSL examples to the prompt
        for example in fsl_examples:
            messages.append({"role": "user", "content": f"Text: {example['final_submission']}\nIs this text Human, Hybrid, or AI?"})
            messages.append({"role": "assistant", "content": example['label_type']})

        messages.append({"role": "user", "content": f"Text: {submission_text}\nIs this text Human, Hybrid, or AI?"})

        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=messages
            )
            classification = response.choices[0].message.content
            submission['fsl_ai_detection'] = classification
        except Exception as e:
            print(f"Error classifying AI detection submission: {e}")
            submission['fsl_ai_detection'] = f"Error classifying: {e}"

display(data)

{'accounting': {'domain': 'Accounting',
  'prompt': 'Analyze the impact of blockchain technology on traditional accounting practices, including auditing and financial reporting.',
  'rubric': {'rubric_id': 'rub_accounting_0001',
   'criteria': [{'criterion_id': 'c1',
     'name': 'Understanding of Blockchain Concepts',
     'description': 'Demonstrates knowledge of blockchain principles (decentralization, immutability, smart contracts) and their relevance to accounting.',
     'performance_descriptors': {'excellent': 'Clearly explains multiple blockchain concepts and maps each to relevant accounting applications.',
      'good': 'Accurately describes 1–2 blockchain concepts and gives examples.',
      'average': 'Mentions blockchain concepts but with limited detail.',
      'needs_improvement': 'Basic or unclear understanding of concepts.',
      'poor': 'Fails to explain blockchain accurately.'}},
    {'criterion_id': 'c2',
     'name': 'Analysis of Accounting Impact',
     'descripti

In [18]:
import json

output_data = []

if data:
    for domain, domain_data in data.items():
        if 'submissions' in domain_data:
            for submission in domain_data['submissions']:
                output_entry = {
                    'domain': domain,
                    'original_submission': submission.get('final_submission'),
                    'ground_truth_label': submission.get('label_type'), # Added ground truth label
                    'zsl_feedback': submission.get('zsl_feedback'),
                    'fsl_ai_detection': submission.get('fsl_ai_detection')
                }
                output_data.append(output_entry)

output_filename = "model_outputs.json"

with open(output_filename, 'w') as f:
    json.dump(output_data, f, indent=4)

print(f"Outputs saved to {output_filename}")

Outputs saved to model_outputs.json
